In [1]:
import tensorflow as tf
import numpy as np

In [48]:
class VGG16:
    
    def __init__(self, img):
        
        #predefining functions and variables
        self.convlayers()
        self.fc_layers()
        
        #.. not needed
        self.setup_kernel()
        self.setup_conv()
        self.setup_bias()
        #.. **
        
        self.img = img
        
        
    
    #may not be useful..
    #....,,, starting from here ,,,....
    def setup_kernel(self, prev_ch, next_ch, K_name):
        return tf.Variable(initializer=tf.truncated_normal(shape=[3, 3, prev_ch, next_ch], mean=0.0, stddev=1.0,
                                                               seed = 1, name=K_name))
    
    def setup_conv(self, input_, kernel):
        return tf.nn.conv2d(input_, kernel, strides=[1,1,1,1], padding='SAME')
    
    def setup_bias(self, next_ch, b_name):
        return tf.get_variable(tf.zeros(next_ch), dtype=tf.float32, name = b_name)
    #....,,, ending here ,,,....
    
    
    #function for conv layer
    def convlayers(self, curr_layer_str, input_ch, next_ch, conv_ip):
        
        #convlayer 1 kernel = (3,3), input channel = 3, output hannel = 64, 
        with tf.name_scope(curr_layer_str) as scope:
            kernel = tf.Variable(initializer=tf.truncated_normal(shape=[3, 3, input_ch, next_ch], mean=0.0, 
                                                                 stddev=1.0, name= 'Weights'))
            #kernel  = self.setup_kernel(input_ch, next_ch, 'Weights')
            
            conv = tf.nn.conv2d(conv_ip, kernel, strides=[1,1,1,1], padding='SAME')
            #conv = self.setup_conv(conv_ip, kernel)
            
            bias = tf.get_variable(tf.zeros(next_ch), dtype=tf.float32, name = 'bias')
            #bias = self.setup_bias(next_ch, 'bias')
            
            out = tf.nn.bias_add(conv, bias)
            return tf.nn.relu(out, name = scope)
    
    #function for max pooling layer
    def pooll(self, pool_ip, pool_name): #pool layer.. pooll
        return tf.nn.max_pool(pool_ip, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name = pool_name)
    
    #function for fully connected layer
    def fc_layers(fc_ip, ip_shape, op_shape, fc_name):
        with tf.name_scope(fc_name) as scope:
            fcw = tf.Variable(tf.truncated_normal([ip_shape, op_shape], mean = 0, stddev = 1.0, name='Weights'))
            fcb = tf.Variable(tf.constant(1.0, shape = op_shape, name = 'bias'))
            
            fc = tf.nn.bias_add(tf.matmul(fc_ip, fcw), fcb)
            
            return tf.nn.relu(fc)
    
    
    #VGG architecture
    def VGG(self):
        
        #1st layer has two convolution functions (total = 2)
        #input channels = 3
        #output channels = 64
        #no of layers = 2
        self.conv1_1 = self.convlayers('conv1_1', 3, 64, self.img)
        self.conv1_2 = self.convlayers('conv1_2', 64, 64, self.convl_1)  
        #max pool layer
        self.pool_1 = self.pooll(self.convl_2, 'pool_1')
        
        #2nd layer has two convolution functions (total = 4)
        #input channels = 64
        #output channels = 128
        #no of layers = 2
        self.conv2_1 = self.convlayers('conv2_1', 64, 128, self.pool_1)
        self.conv2_2 = self.convlayers('conv2_2', 128, 128, self.conv2_1)
        #max pool layer
        self.pool_2 = self.pooll(self.conv2_2, 'pool_2')
        
        #3rd layer has three convolution functions (total = 7)
        #input channel = 128
        #output channels = 256
        #no of layers = 3
        self.conv3_1 = self.convlayers('conv3_1', 128, 256, self.pool_2)
        self.conv3_2 = self.convlayers('conv3_2', 256, 256, self.conv3_1)
        self.conv3_3 = self.convlayers('conv3_3', 256, 256, self.conv3_2)
        #max pool layer
        self.pool_3 = self.pool(self.conv3_3, 'pool_3')
        
        #4th layer has three convolution functions (total = 10)
        #input channels = 256
        #output channels = 512
        #no of layers = 3
        self.conv4_1 = self.convlayers('conv4_1', 256, 512, self.pool_3)
        self.conv4_2 = self.convlayers('conv4_2', 512, 512, self.conv4_1)
        self.conv4_3 = self.convlayers('conv4_3', 512, 512, self.conv4_2)
        #max pool layer
        self.pool_4 = self.pool(self.conv4_3, 'pool_4')
        
        #5th layer has three convolution functions (total = 13)
        #input channels = 512
        #output channels = 512
        #no of layers = 3
        self.conv5_1 = self.convlayers('conv5_1', 512, 512, self.pool_4)
        self.conv5_2 = self.convlayers('conv5_2', 512, 512, self.conv5_1)
        self.conv5_3 = self.convlayers('conv5_3', 512, 512, self.conv5_2)
        #max pool layer
        self.pool_5 = self.pool(self.conv5_3, 'pool_5')
        
        #flattening... please go through "example for reshape"
        shape = tf.shape(self.pool_5)
        self.pool_5_flat = tf.reshape(self.pool_5, [-1, 7*7*512])
        
        #6th layer has one computation (total = 14)
        #fully connected layer 1 .. 4096
        self.fc1 = self.fc_layers(self.pool_5_flat, shape[1]*shape[2]*shape[3], 4096, 'fc1')
        #adding dropouts
        #self.fc1d = tf.nn.dropout(fc1, keep_prob=0.5)
        
        #7th layer has one computation (total = 15)
        #fully connected layer 2 .. 4096
        self.fc2 = self.fc_layers(self.fc1, 4096, 4096, 'fc2')
        #adding dropouts
        #self.fc2d = tf.nn.dropout(fc2, keep_prob=0.5)
        
        #8th layer has one computation (total = 16)... hence VGG 16
        #fully connected layer 3 .. 1000
        fcw = tf.Variable(tf.truncated_normal([4096, 1000], mean = 0, stddev = 1.0, name='fc5_Weights'))
        fcb = tf.Variable(tf.constant(1.0, shape=1000, name = 'fc5_bias'))
            
        self.fc3 = tf.nn.bias_add(tf.matmul(self.fc2, fcw), fcb)
        
        #self.prob = tf.nn.softmax(self.fc3, name = 'prob')
        
        print("build model finished")
        
        #return self.fc3
        #return self.prob

In [42]:
# example for reshape
import numpy as np
x = tf.Variable(tf.truncated_normal(shape=[4,3,3,2], mean = 0, stddev=1.0))
init = tf.global_variables_initializer()

shape = tf.shape(x)
y = tf.reshape(x, [-1,shape[1]*shape[2]*shape[3]])

sess = tf.Session()
sess.run(init)
print(sess.run([shape, tf.shape(y), shape[1:], shape[1]*shape[2]*shape[3]]))

[array([4, 3, 3, 2]), array([ 4, 18]), array([3, 3, 2]), 18]


In [ ]:
def data_preprocess(x):
    x = tf.convert_to_tensor(x, tf.float32)
    mean, variance = tf.nn.moments(x, axes=[0,1,2])
    x_normalized = tf.nn.batch_norm_with_global_normalization(t=x, m = mean, v= variance, beta=None, 
                                                                gamma=None, variance_epsilon=1e-7, 
                                                                scale_after_normalization=None)
    return x_normalized.eval()

In [49]:
x = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.float32, [None, 10])

vgg = VGG16(x)
prob = vgg.fc3

#we need to pass one hot encoded y for softmax_cross_entropy
#when using sparse_softmax_cross_entropy, no need of one hot encoding of y
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=prob, labels = y))
train_operation = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)


correct_pred = tf.equal(tf.argmax(prob, 1), y)
accuracy_operation = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
      
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

import time

with tf.Session() as sess:
  
  sess.run(tf.global_variables_initializer())
  num_examples = len(x_train)
  
  print("Training....\n")
  print("For Batch size...", BATCH_SIZE)
  
  for i in range(EPOCHS):
    start = time.process_time()
    
    for offset in range(0, num_examples, BATCH_SIZE):
      end = offset + BATCH_SIZE

      batch_x, batch_y = x_train[offset:end], y_train[offset:end]
      sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
      
    

    validation_accuracy = evaluate(x_val, y_val)
    training_accuracy = evaluate(x_train, y_train)
    print("EPOCH {} ...".format(i+1))
    print("Validation Accuracy = {:.3f}".format(validation_accuracy))
    print("Training Accuracy = {:.3f}".format(training_accuracy))
    print("Time taken by Epoch: ",i ,time.process_time() - start)
    print()

TypeError: convlayers() missing 4 required positional arguments: 'curr_layer_str', 'input_ch', 'next_ch', and 'conv_ip'